In [ ]:
#Connecting the google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
#defining the train directory
train_dir='xxxxx'

In [ ]:
#importing the necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
import cv2
import warnings
warnings.filterwarnings('ignore')
import os

In [ ]:
#Making a list with class label(name of plant) and file path in it
plant = []
plant_image = []
for i in os.listdir(train_dir):
    for image_filename in os.listdir(train_dir + i):
        plant.append(i) # name of the plant
        plant_image.append(i + '/' + image_filename)

In [ ]:
#making a dataframe
train_plants = pd.DataFrame(plants, columns=["plants"])
train_plants["plant Image"] = plants_image

In [ ]:
#initialising 2 lists to store the path and class label
image_paths=[]
data_labels=[]

In [ ]:
#iterating over the previously created dataframe
for i in range(len(train_plants)):
  path=train_plants['plants Image'][i]
  path=train_dir+path
  label=train_plants['plants'][i]
  image_paths.append(path)
  data_labels.append(label)

In [ ]:
# For storing the class labels for Training and Validation datasets
train_data_class_list = []
val_data_class_list = []

# For storing the paths of images for Training and Validation datasets
train_data_path_list = []
val_data_path_list = []

In [ ]:
# Training and Validation split
from sklearn.model_selection import train_test_split
(train_data_path_list, val_data_path_list, train_data_class_list, val_data_class_list) = train_test_split(train_data_path_list,
                                                                                                        train_data_class_list,
                                                                                                        test_size = 0.285,
                                                                                                        stratify = train_data_class_list,
                                                                                                        random_state = 101)

In [ ]:
len(train_data_path_list)

In [ ]:
len(val_data_path_list)

In [ ]:
#making a dataframe for train and validation
df_train = pd.DataFrame({"Path":train_data_path_list,"Class":train_data_class_list})
df_val = pd.DataFrame({"Path":val_data_path_list,"Class":val_data_class_list})

In [ ]:
df_train

In [ ]:
print(len(df_train))
print(len(df_val))

In [ ]:
len(pd.unique(df_train['Class']))

In [ ]:
len(pd.unique(df_val['Class']))

In [ ]:
#using the pre defined function to initialise the class weights
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',classes=pd.unique(df_train['Class']),y=train_data_class_list)

In [ ]:
class_weights

In [ ]:
class_weights = dict(enumerate(class_weights))

In [ ]:
# Initialize the initial values of different Hyperparameters
INIT_LR = 0.0001        # learning rate
EPOCHS = 5            # epochs
BS = 32                # batch size
DROPOUT = 0.5          # value of dropout
MAXPOOL_SIZE = (2, 2)  # pool window size for Average Pooling Layer
ROTATION_DEG = 15      # rotation degree for Data Augmentation

In [ ]:
#importing libraries to train the model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
#importing the base model as VGG 16
baseModel = VGG16(weights = "imagenet", include_top = False, input_tensor = Input(shape=(100, 100, 3)))

In [ ]:
#defining the head model over the base model
headModel = baseModel.output
headModel = Flatten(name = "flatten")(headModel)
headModel = Dense(units = 128, activation = "relu")(headModel)
headModel = Dropout(DROPOUT)(headModel)
headModel = Dense(131, activation = "softmax")(headModel)

In [ ]:
# Place the head FC model on top of the base model
model = Model(inputs = baseModel.input, outputs = headModel)

for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
#initialising the optimizer and compiling the model
opt = Adam(lr = INIT_LR, decay = INIT_LR / EPOCHS)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy', tf.keras.metrics.Precision(name="precision"), tf.keras.metrics.Recall(name="recall"), tf.keras.metrics.AUC(name="auc"),
                                                                           tf.keras.metrics.TruePositives(name="tp"),
                                                                           tf.keras.metrics.FalsePositives(name="fp"),
                                                                           tf.keras.metrics.TrueNegatives(name="tn"),
                                                                           tf.keras.metrics.FalseNegatives(name="fn")])

In [ ]:
tr_data = ImageDataGenerator(rotation_range = ROTATION_DEG, horizontal_flip = True, fill_mode = "nearest")
val_data = ImageDataGenerator(rotation_range = ROTATION_DEG, horizontal_flip = True, fill_mode = "nearest")

training_df = tr_data.flow_from_dataframe(df_train, x_col = "Path",y_col = "Class", batch_size = BS, target_size = (100, 100))
validation_df = val_data.flow_from_dataframe(df_val, x_col = "Path",y_col = "Class", batch_size = BS, target_size = (100, 100))

In [ ]:
#definining the directory location where the models and the logs will be stored
ModelDir = "yyyyyy"

LogDir = "zzzzzzz"

In [ ]:
import time
LogPath = time.strftime("%Y%m%d-%H%M", time.gmtime()) + "-vgg16"

In [ ]:
# checkpointing
from tensorflow import keras
csv_logger = keras.callbacks.CSVLogger(LogDir + "/" + LogPath + "-acc.csv", append = True)

entirecheckpointLast = keras.callbacks.ModelCheckpoint(filepath = ModelDir + "/" + LogPath + "-entire-last.h5", verbose = 0)

entirecheckpointBest = keras.callbacks.ModelCheckpoint(filepath = ModelDir + "/" + LogPath + "-entire-best.h5", verbose = 1, save_best_only = True)

In [ ]:
#trainin the model
History = model.fit(training_df,
                 steps_per_epoch = len(df_train) // BS,
                 validation_data = validation_df,
                 validation_steps = len(df_val) // BS,
                 epochs = EPOCHS,
                 class_weight = class_weights,
                 callbacks = [csv_logger, entirecheckpointLast, entirecheckpointBest]
                 )

In [ ]:
#Saving the model
model_path = "sssss"
model.save(model_path)